In [1]:
%load_ext autoreload
%autoreload 2
import time
import multiprocessing
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.option as option
import thor.track as track
import thor.analyze as analyze
import thor.parallel as parallel
import thor.visualize as visualize
import thor.log as log

notebook_name = "gridrad_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
year=2010
event_directories = data.gridrad.get_event_directories(year, base_local=base_local)
event_directory = event_directories[0]
start, end, event_start = data.gridrad.get_event_times(event_directory)
# start = "2010-01-21T00:30:00"
# end = "2010-01-21T01:30:00"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / f"runs/dev/gridrad_{event_start.replace('-', '')}"
event_start

'2010-01-20'

In [3]:
# if output_parent.exists():
    # shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create the data_options dictionary
converted_options = {"save": False, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options, event_start=event_start
)
lon_range = [-102, -89]
lat_range = [27, 39]
era5_pl_options = data.era5.data_options(
    start=start, end=end, longitude_range=lon_range, latitude_range=lat_range
)
args_dict = {"start": start, "end": end, "data_format": "single-levels"}
args_dict.update({"longitude_range": lon_range, "latitude_range": lat_range})
era5_sl_options = data.era5.data_options(**args_dict)

data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)
# for testing, ignore the era5 data
data_options = option.consolidate_options([gridrad_options])

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory=options_directory)
gridrad_options = data_options["gridrad"]

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory=options_directory)

# Create the track_options dictionary
track_options = option.default_track_options(dataset="gridrad")
# Modify the default options for gridrad. Because grids so large we now use a distinct
# global flow box for each object.
track_options.levels[1].objects[0].tracking.global_flow_margin = 70
track_options.levels[1].objects[0].tracking.unique_global_flow = False
# If testing, remove the profile and tag attributes
track_options.levels[1].objects[0].attributes["mcs"].pop("profile")
track_options.levels[1].objects[0].attributes["mcs"].pop("tag")
track_options.to_yaml(options_directory / "track.yml")

# Create the display_options dictionary
# visualize_options = {
#     obj: visualize.option.runtime_options(obj, save=True, style="presentation")
#     for obj in ["mcs"]
# }
visualize_options = None

2024-10-31 10:58:47,640 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.


In [8]:
%load_ext memory_profiler

In [11]:
times = data.utils.generate_times(data_options["gridrad"])
args = [times, data_options.copy(), grid_options.copy()]
args += [track_options.model_copy(), visualize_options]
track.simultaneous_track(*args, output_directory=output_parent)

2024-10-30 17:22:04,070 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/dev/gridrad_20100120.
2024-10-30 17:22:04,072 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-30 17:22:04,167 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-10-30 17:22:04,169 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-10-30 17:22:04,170 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-10-30 17:22:06,082 - thor.track - INFO - Processing hierarchy level 0.
2024-10-30 17:22:06,082 - thor.track - INFO - Tracking convective.
2024-10-30 17:22:06,271 - thor.track - INFO - Tracking middle.
2024-10-30 17:22:06,348 - thor.track - INFO - Tracking anvil.
2024-10-30 17:22:06,405 - thor.track - INFO - Processing hierarchy level 1.
2024-10-30 17:22:06,406 - thor.track - INFO - Tracking mcs.
2024-10-30 17:22:06,612 - thor.match.match - INFO - Matching mcs objects.
2024-10-30 1

KeyboardInterrupt: 

In [12]:
# num_processes = int(0.75 * os.cpu_count())
# num_processes = os.cpu_count()
num_processes = 4
kwargs = {"initializer": parallel.initialize_process, "processes": num_processes}
with log.logging_listener(), multiprocessing.get_context("spawn").Pool(**kwargs) as pool:
    results = []
    for i, time_interval in enumerate(intervals):
        time.sleep(1)
        args = [i, time_interval, data_options.copy(), grid_options.copy()]
        args += [track_options.model_copy(), visualize_options]
        args += [output_parent, "gridrad"]
        args = tuple(args)
        results.append(pool.apply_async(parallel.track_interval, args))
    pool.close()
    pool.join()
    parallel.check_results(results)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2024-10-30 17:28:04,405 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/dev/gridrad_20100120/interval_1.
2024-10-30 17:28:04,408 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-30 17:28:04,436 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/dev/gridrad_20100120/interval_0.
2024-10-30 17:28:04,438 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-30 17:28:04,708 - thor.track - INFO - Processing 2010-01-20T19:00:00.
2024-10-30 17:28:04,710 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T19:00:00.
2024-10-30 17:28:04,711 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T190000Z.nc
2024-10-30 17:28:04,751 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-10-30 17:28:04,752 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-10-30 17:28:04,753 - thor.data.gridrad - INFO - Converting gridrad dat

In [13]:
parallel.stitch_run(output_parent, intervals, cleanup=False)

2024-10-30 17:38:27,355 - thor.parallel - INFO - Stitching all attribute, mask and record files.
2024-10-30 17:38:39,098 - thor.parallel - INFO - Stitching record files.
2024-10-30 17:38:39,229 - thor.parallel - INFO - Stitching attribute files.
2024-10-30 17:38:42,878 - thor.parallel - INFO - Stitching mask files.


In [40]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_parent, profile_dataset=None)
analyze.mcs.quality_control(output_parent, analysis_options)
# analyze.mcs.classify_all(output_parent)

convective_contained  anvil_contained  \
time                universal_id                                          
2010-01-20 18:00:00 1                             True             True   
                    2                             True             True   
2010-01-20 18:10:00 1                             True             True   
                    2                             True             True   
2010-01-20 18:20:00 1                             True             True   
...                                                ...              ...   
2010-01-21 03:10:00 15                            True             True   
                    31                            True             True   
                    36                            True             True   
2010-01-21 03:20:00 15                            True             True   
                    31                            True             True   

                                  velocity  area  offset  major_axis  \
time                universal_id                                       
2010-01-20 18:00:00 1                 True  True    True        True   
                    2                 True  True    True        True   
2010-01-20 18:10:00 1                 True  True    True        True   
                    2                 True  True   False        True   
2010-01-20 18:20:00 1                 True  True    True        True   
...                                    ...   ...     ...         ...   
2010-01-21 03:10:00 15                True  True    True        True   
                    31                True  True    True        True   
                    36                True  True   False        True   
2010-01-21 03:20:00 15                True  True    True        True   
                    31                True  True    True        True   

                                  axis_ratio  duration  parents  
time                universal_id                                 
2010-01-20 18:00:00 1                   True      True    False  
                    2                  False     False    False  
2010-01-20 18:10:00 1                   True      True    False  
                    2                  False     False    False  
2010-01-20 18:20:00 1                  False      True    False  
...                                      ...       ...      ...  
2010-01-21 03:10:00 15                  True      True     True  
                    31                 False      True    False  
                    36                  True      True     True  
2010-01-21 03:20:00 15                  True      True     True  
                    31                 False      True    False  

[273 rows x 9 columns]

In [4]:
figure_name = f"mcs_gridrad_{event_start.replace('-', '')}"
figure_options = visualize.option.horizontal_attribute_options(
    figure_name, style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64(start)
end_time = np.datetime64(end)
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 7200, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)

2024-10-31 10:58:49,079 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:00:00.000000000.
2024-10-31 10:58:51,857 - thor.visualize.attribute - INFO - Saving mcs_gridrad_20100120 figure for 2010-01-20T18:00:00.000000000.



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2024-10-31 10:58:58,160 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:10:00.000000000.
2024-10-31 10:58:58,312 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:20:00.000000000.
2024-10-31 10:58:59,881 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:30:00.000000000.
2024-10-31 10:59:01,876 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:40:00.000000000.
2024-10-31 10:59:02,362 - thor.visualize.attribute - INFO - Saving mcs_gridrad_20100120 figure for 2010-01-20T18:10:00.000000000.
2024-10-31 10:59:02,809 - thor.visualize.attribute - INFO - Saving mcs_gridrad_20100120 figure for 2010-01-20T18:20:00.000000000.
2024-10-31 10:59:04,361 - thor.visualize.attribute - INFO - Saving mcs_gridrad_20100120 figure for 2010-01-20T18:30:00.000000000.
2024-10-31 10:59:05,232 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:50:00.000000000.
2024-10-31 10:59:05,658 - thor.visu